In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import sys
sys.path.append("..")

from utils.tools import randomSplit

/home/kelon/anaconda3/envs/CVD/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
print(os.getcwd())
# modify
ts_data = 'P18'

# modify
text_data = 'Discharge_summary'

save_dir = ts_data + '_' + text_data + '/'

raw_path = save_dir + 'rawdata/'
text_path = raw_path + text_data + '.csv'

/home/kelon/code/CVD/Multi_data


In [3]:
df_ts = pd.read_csv(raw_path + 'biomarks_plus.csv', low_memory=False)
df_text = pd.read_csv(text_path, low_memory=False)
df_label = pd.read_csv(raw_path + 'Outcomes.csv', sep=",", header=0,
                        names=["RecordID","PatientID","Length_of_stay","Death","Cvd", "ICD"])

In [4]:
ts_params = df_ts.Parameter.unique()
ts_params = list(ts_params)
ts_params = np.array(ts_params)

In [5]:
# modify according to time series dataset
static_params = ['Age','Gender','Ethnicity','Marital']

In [6]:
# 保存时序变量和静态变量
np.save(save_dir + 'ts_params.npy', ts_params)
print('ts_params.npy: the names of variables')

np.save(save_dir + 'static_params.npy', static_params)
print('save names of static descriptors: static_params.npy')

ts_params.npy: the names of variables
save names of static descriptors: static_params.npy


In [7]:
def parse_all(path, num):
    # num is the number of static parameters
    P_list = []
    cnt = 0
    allfiles = os.listdir(path)
    allfiles.sort()
    for f in tqdm(allfiles):
        file_name, file_ext = os.path.splitext(f)
        if file_ext == '.txt':
            df = pd.read_csv(path+file_name+'.txt', sep=",", header=1, names=["time", "param", "value"])
            df_demogr = df.iloc[0:num]
            # print(df_demogr)
            df_data = df.iloc[num:]
            
            arr_demogr = np.array(df_demogr)
            arr_data = np.array(df_data)

            my_dict = {'id': file_name, 'static':[]}
            for i in range(num):
                my_dict['static'].append(int(arr_demogr[i,2]))

            # time-series
            my_dict['ts'] = arr_data
            
            # append patient dictionary in master dictionary
            P_list.append(my_dict)
            cnt += 1
    return P_list

In [8]:
P_list = parse_all(raw_path + 'set/', len(static_params))

100%|██████████| 56857/56857 [01:31<00:00, 619.63it/s]


In [9]:
# judge if P_list['id'] is sorted
arr = []
for item in P_list:
    arr.append(int(item['id']))
assert arr == sorted(arr)

In [10]:
np.save(save_dir + 'P_list.npy', P_list)

In [11]:
label_set = set(df_label.RecordID.unique())
ts_set = set(df_ts.RecordID.unique())
text_set = set(df_text.RecordID.unique())
inter_set = label_set & ts_set & text_set

In [12]:
len(inter_set)

51176

In [13]:
# filter labels
df_label.set_index('RecordID', inplace=True)
df_label = df_label.loc[inter_set]
df_label.reset_index(inplace=True)
df_label.sort_values(['RecordID'], ascending=True, inplace=True)

In [14]:
df_label.head()

,RecordID,PatientID,Length_of_stay,Death,Cvd,ICD
35220,100001,58526,149,0,0,NaN
35221,100003,54610,98,0,0,NaN
35222,100006,9895,289,0,0,NaN
35223,100007,23018,175,0,0,NaN
35224,100009,533,118,0,1,414.8-411.1-414.01-414.2


In [15]:
df_label['ICD'] = df_label['ICD'].apply(lambda x: "0" if pd.isnull(x) else x)

In [16]:
# Store outcomes in csv format
df_label.to_csv(save_dir + 'df_outcomes.csv', index=None)

In [17]:
labels = set()
def func(x):
    global labels
    b = set(x.split('-'))
    labels = labels | b
    return x

In [18]:
df_label.ICD.apply(func)

35220                           0
35221                           0
35222                           0
35223                           0
35224    414.8-411.1-414.01-414.2
                   ...           
35215                      410.31
35216                           0
35217                           0
35218                411.1-414.01
35219                      414.01
Name: ICD, Length: 51176, dtype: object

In [19]:
df_icd = pd.DataFrame({'code': list(labels)})

In [20]:
# 可能存在某些待预测的ICD CODE在数据集中无记录，将其删除
df_icd.to_csv(save_dir + 'ICD_code.csv', index=None)

In [21]:
# filter text
df_text.sort_values('RecordID', inplace=True, ascending=True)
df_text.set_index('RecordID', inplace=True)
df_text = df_text.loc[inter_set]
df_text.reset_index(inplace=True)
df_text.sort_values(['RecordID'], ascending=True, inplace=True)

In [22]:
df_text.head()

,RecordID,PatientID,Time,Text
35220,100001,58526,4660773120,11:12 AM CHEST (PA & LAT) ...
35221,100003,54610,5689447200,Sinus rhythm Prolonged QT interval is nonspeci...
35222,100006,9895,4363120800,Sinus tachycardia Left axis deviation -\nanter...
35223,100007,23018,5530212000,Sinus rhythm Atrial premature complex Consider...
35224,100009,533,6070644000,"Sinus bradycardia, Left atrial abnormality, ..."


In [23]:
# filter time series sensors
length = len(P_list)
P_new = []
for i in range(length):
    if int(P_list[i]['id']) not in inter_set:
        continue
    else:
        P_new.append(P_list[i])
P_list = P_new

In [24]:
# Based on visulization.ipynb
max_tmins = 240*60
len_ts = []
MAX_LEN = 500

In [25]:
"""Group all patient time series into arrays"""
n = len(P_list)
F = len(ts_params)
PTdict_list = []

for ind in tqdm(range(n)):
    ID = P_list[ind]['id']
    static = P_list[ind]['static']
    ts = P_list[ind]['ts']
    
    # find unique times
    unq_tmins = []
    i = 0
    for sample in ts:
        current_tmin = sample[2]
        if (current_tmin not in unq_tmins) and (current_tmin < max_tmins) and i < MAX_LEN:
            unq_tmins.append(current_tmin)
            i += 1
    unq_tmins = np.array(unq_tmins)

    # one-hot encoding of categorical static variables
    extended_static = [static[0], 0,0, 0,0,0, 0,0,0,0]
    extended_static[static[1]+1] = 1
    extended_static[static[2]+3] = 1
    extended_static[static[3]+6] = 1
   
    # construct array of maximal size
    Parr = np.zeros((MAX_LEN,F))
    Tarr = np.zeros((MAX_LEN,1))

    # for each time measurement find index and store
    for sample in ts:
        tmins = sample[2]
        param = sample[-2]
        value = sample[-1]
        if tmins < max_tmins:
            if len(np.where(tmins==unq_tmins)[0]) == 0:
                break
            time_id = np.where(tmins==unq_tmins)[0][0]
            param_id = np.where(ts_params==param)[0][0]
            Parr[time_id, param_id] = value
            Tarr[time_id, 0] = unq_tmins[time_id]
    
    length = len(unq_tmins)
    
    # construct dictionary
    my_dict = {'id':ID, 'static':static, 'extended_static':extended_static, 'arr':Parr, 'time':Tarr, 'length':length}
    
    # add array into list
    PTdict_list.append(my_dict)

100%|██████████| 51176/51176 [05:34<00:00, 153.00it/s]


In [26]:
df_text.to_csv(save_dir + text_data + '.csv', index=None)
np.save(save_dir + 'PTdict_list.npy', PTdict_list)

In [30]:
# randomSplit(n, save_dir + 'splits/', K_splits = 5, p_train=0.9, p_val=0.05)

/home/kelon/anaconda3/envs/CVD/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [27]:
# 显示划分id
df = pd.read_csv(save_dir + text_data + '.csv')
for i in range(1, 6):
    train, dev, test = np.load(save_dir + 'splits/%d_fold.npy'%(i), allow_pickle=True)
    df_train = df.RecordID.iloc[train]
    df_dev = df.RecordID.iloc[dev]
    df_test = df.RecordID.iloc[test]

    df_train.to_csv(save_dir + 'train_ids_fold_%d.csv'%(i), index=None)
    df_dev.to_csv(save_dir + 'dev_ids_fold_%d.csv'%(i), index=None)
    df_test.to_csv(save_dir + 'test_ids_fold_%d.csv'%(i), index=None)

In [11]:
train, dev, test = np.load(save_dir + 'splits/1_fold.npy', allow_pickle=True)

In [10]:
save_dir

'P18_Discharge_summary/'

In [13]:
len(train)

46036

In [12]:
len(dev)

2558

In [14]:
2558*18

46044